In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8051"

In [ ]:
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

def research_messages_hook(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

research_message_listener = {
    "handler": research_messages_hook,
    "topic": "basicmessages"
}



In [7]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [8]:
agent_controller.register_listeners([research_message_listener], defaults=True)

In [9]:
#Paste in invitation from data_owner agent
invite = {'connection_id': '1ef4764f-b40c-498f-a8ae-ead9637fbdd9', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '177c7dfb-2adc-4f06-a306-dfae75c6574e', 'label': 'Bob', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMTc3YzdkZmItMmFkYy00ZjA2LWEzMDYtZGZhZTc1YzY1NzRlIiwgImxhYmVsIjogIkJvYiIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJyZWNpcGllbnRLZXlzIjogWyI0ZjhBNXJVS0xYRk1IUHR4WENNWk42TUNXRHNTQUI0ZDNUQzdVTWNmNFJlZSJdfQ=='}




In [10]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)
# Print out accepted Invite and Researcher ID
print("Connection", response)
researcher_id = response["connection_id"]


ACCEPTED CONNECTION -  {'invitation_mode': 'once', 'my_did': '2Qbiw2yuNDpxtX8CHwKCze', 'state': 'request', 'request_id': 'aa515ad5-8f30-413d-b5fa-cf890c37657c', 'accept': 'manual', 'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'invitation_key': '4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree', 'their_label': 'Bob', 'updated_at': '2020-06-25 15:10:19.618431Z', 'initiator': 'external', 'routing_state': 'none', 'created_at': '2020-06-25 15:10:19.599577Z'}
{'invitation_mode': 'once', 'my_did': '2Qbiw2yuNDpxtX8CHwKCze', 'state': 'request', 'request_id': 'aa515ad5-8f30-413d-b5fa-cf890c37657c', 'accept': 'manual', 'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'invitation_key': '4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree', 'their_label': 'Bob', 'updated_at': '2020-06-25 15:10:19.618431Z', 'initiator': 'external', 'routing_state': 'none', 'created_at': '2020-06-25 15:10:19.599577Z'}
Connection {'invitation_mode': 'once', 'my_did': '2Qbiw2yuNDpxtX8CHwKCze', 'state': 'reques

In [11]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA
{'thread_id': '3bf126bd-66dd-44df-8703-436a739c4b21'}


In [12]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

RESEARCH AGENT CONNECTION
{'invitation_mode': 'once', 'my_did': '2Qbiw2yuNDpxtX8CHwKCze', 'their_did': '48RxqgaB7ywbPqMpTqxg2Z', 'state': 'active', 'request_id': 'aa515ad5-8f30-413d-b5fa-cf890c37657c', 'accept': 'manual', 'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'invitation_key': '4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree', 'their_label': 'Bob', 'updated_at': '2020-06-25 15:10:42.639789Z', 'initiator': 'external', 'routing_state': 'none', 'created_at': '2020-06-25 15:10:19.599577Z'}


In [16]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)

Handle research messages  {'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'message_id': '15ede282-9a90-485c-b542-3ff3fda17f64', 'content': 'hello from data owner world! WOOPWOOP', 'state': 'received'} 82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390
Handle research messages  {'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'message_id': '8ac4ed37-cebe-4cbc-9ac2-567285a60dc1', 'content': 'THIS IS MY RESPONSE - DATA AGENT', 'state': 'received'} 82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390
Handle research messages  {'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'message_id': '6d70e70c-4fa5-4e42-8ca1-e022e28c4ad9', 'content': 'THIS IS MY RESPONSE - DATA AGENT', 'state': 'received'} 82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390
Handle research messages  {'connection_id': '82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390', 'message_id': 'dfd39131-76f8-430e-8817-456b4465308a', 'content': 'THIS IS MY RESPONSE - DATA AGENT', 'state': 'received'} 82ac9c5c-8fe6-4dc0-b4fa-c2e7a4e0d390
BASIC MESSAGE - RESEARC